# Examples of utils provided by ReGraph's Neo4jHierarchy module

In [1]:
import networkx as nx
from regraph import Rule, plot_rule


from regraph.neo4j.hierarchy import Neo4jHierarchy
import regraph.neo4j.cypher_utils as cypher
from regraph.neo4j.category_utils import _check_consistency, _check_rhs_consistency

## Initializing Neo4j database

0. When installing neo4j you will be asked to choose login/password for you dbs (here its "neo4j"/"admin"), if you choose other -- change in the cell below.
1. To start neo4j server run `sudo service neo4j start`
2. Check status by running `sudo service neo4j status`. Here you can check the _bolt_ port, change the cell below if different from 7687
3. You can query the db by using the neo4j browser, the address can be found also in the result of 'status', e.g. "Remote interface available at http://localhost:7474/".

In [2]:
# initialize the neo4j driver, wrappped into Neo4jHierarchy object
h = Neo4jHierarchy(uri="bolt://localhost:7687", user="neo4j", password="admin")
h.clear()

### Definition of the meta-model

In [3]:
nodes = [
    ("Gene", {"type": {"Gene"}}),
    ("Region", {"type": {"Region"}}),
    ("Residue", {"type": {"Residue"}}),
    ("Locus", {"type": {"Locus"}}),
    ("State", {"type": {"State"}, "state": {"True", "False"}}),
    ("mod", {"type": {"mod"}}),
    ("bnd", {"type": {"bnd"}})
]
edges = [
    ("Gene", "mod"),
    ("Gene", "Locus"),
    ("Region", "Gene"),
    ("Region", "Locus"),
    ("Region", "mod"),
    ("Residue", "Gene"),
    ("Residue", "Region"),
    ("bnd", "Locus"),
    ("State", "Gene"),
    ("State", "Region"),
    ("State", "Residue"),
    ("mod", "State")
]
h.add_graph('metaModel', nodes, edges)

### Definition of nuggets

In [4]:
nodes = [
    ("S90", {"type": {"Residue"}}),
    ("Grb2", {"type": {"Gene"}}),
    ("SH2", {"type": {"Region"}}),
    ("Grb2_locus", {"type": {"Locus"}}),
    ("bnd", {"type": {"bnd"}}),
    ("Y1092", {"type": {"Residue"}}),
    ("EGFR", {"type": {"Gene"}}),
    ("EGFR_locus", {"type": {"Locus"}}),
    ("phospho", {"type": {"State"}, "state":{"True"}})
]
edges = [
    ("S90", "Grb2"),
    ("SH2", "Grb2"),
    ("SH2", "Grb2_locus"),
    ("bnd", "Grb2_locus"),
    ("bnd", "EGFR_locus"),
    ("EGFR", "EGFR_locus"),
    ("Y1092", "EGFR"),
    ("phospho", "Y1092")
]
h.add_graph('nugget_1', nodes, edges)

In [5]:
nodes = [
    ("S90", {"type": {"Residue"}}),
    ("Grb2", {"type": {"Gene"}}),
    ("SH2", {"type": {"Region"}}),
    ("Grb2_locus", {"type": {"Locus"}}),
    ("bnd", {"type": {"bnd"}}),
    ("Shc", {"type": {"Gene"}}),
    ("Shc_locus", {"type": {"Locus"}}),
    ("phospho", {"type": {"State"}, "state":{"True"}})
]
edges = [
    ("S90", "Grb2"),
    ("SH2", "Grb2"),
    ("SH2", "Grb2_locus"),
    ("bnd", "Grb2_locus"),
    ("bnd", "Shc_locus"),
    ("Shc", "Shc_locus"),
    ("phospho", "Shc")
]
h.add_graph('nugget_2', nodes, edges)

### Definition of action graph

In [6]:
nodes = [
    ("S90", {"type": {"Residue"}}),
    ("Grb2", {"type": {"Gene"}}),
    ("SH2", {"type": {"Region"}}),
    ("Grb2_locus_1", {"type": {"Locus"}}),
    ("bnd_1", {"type": {"bnd"}}),
    ("Y1092", {"type": {"Residue"}}),
    ("EGFR", {"type": {"Gene"}}),
    ("EGFR_locus", {"type": {"Locus"}}),
    ("phospho_1", {"type": {"State"}, "state":{"True"}}),
    ("Grb2_locus_2", {"type": {"Locus"}}),
    ("bnd_2", {"type": {"bnd"}}),
    ("Shc", {"type": {"Gene"}}),
    ("Shc_locus", {"type": {"Locus"}}),
    ("phospho_2", {"type": {"State"}, "state":{"True"}})
]
edges = [
    ("S90", "Grb2"),
    ("SH2", "Grb2"),
    ("SH2", "Grb2_locus_1"),
    ("bnd_1", "Grb2_locus_1"),
    ("bnd_1", "EGFR_locus"),
    ("EGFR", "EGFR_locus"),
    ("Y1092", "EGFR"),
    ("phospho_1", "Y1092"),
    ("SH2", "Grb2_locus_2"),
    ("bnd_2", "Grb2_locus_2"),
    ("bnd_2", "Shc_locus"),
    ("Shc", "Shc_locus"),
    ("phospho_2", "Shc")
]
h.add_graph('actionGraph', nodes, edges)

### Add typings

In [7]:
mapping = {
    "S90":"Residue",
    "Grb2":"Gene",
    "SH2":"Region",
    "Grb2_locus_1":"Locus",
    "bnd_1":"bnd",
    "Y1092":"Residue",
    "EGFR":"Gene",
    "EGFR_locus":"Locus",
    "phospho_1":"State",
    "Grb2_locus_2":"Locus",
    "bnd_2":"bnd",
    "Shc":"Gene",
    "Shc_locus":"Locus",
    "phospho_2":"State"
}
h.add_typing('actionGraph', 'metaModel', mapping)

In [8]:
mapping = {
    "S90":"S90",
    "Grb2":"Grb2",
    "SH2":"SH2",
    "Grb2_locus":"Grb2_locus_1",
    "bnd":"bnd_1",
    "Y1092":"Y1092",
    "EGFR":"EGFR",
    "EGFR_locus":"EGFR_locus",
    "phospho":"phospho_1"    
}
print(mapping)
h.add_typing('nugget_1', 'actionGraph', mapping)

{'S90': 'S90', 'Grb2': 'Grb2', 'SH2': 'SH2', 'Grb2_locus': 'Grb2_locus_1', 'bnd': 'bnd_1', 'Y1092': 'Y1092', 'EGFR': 'EGFR', 'EGFR_locus': 'EGFR_locus', 'phospho': 'phospho_1'}


In [9]:
mapping = {
    "S90":"S90",
    "Grb2":"Grb2",
    "SH2":"SH2",
    "Grb2_locus":"Grb2_locus_2",
    "bnd":"bnd_2",
    "Shc":"Shc",
    "Shc_locus":"Shc_locus",
    "phospho":"phospho_2"
}
h.add_typing('nugget_2', 'actionGraph', mapping)

In [10]:
mapping = {
    "S90":"Residue",
    "Grb2":"Gene",
    "SH2":"Region",
    "Grb2_locus":"Locus",
    "bnd":"bnd",
    "Y1092":"Residue",
    "EGFR":"Gene",
    "EGFR_locus":"Locus",
    "phospho":"State"    
}
print(mapping)
h.add_typing('nugget_1', 'metaModel', mapping)

{'S90': 'Residue', 'Grb2': 'Gene', 'SH2': 'Region', 'Grb2_locus': 'Locus', 'bnd': 'bnd', 'Y1092': 'Residue', 'EGFR': 'Gene', 'EGFR_locus': 'Locus', 'phospho': 'State'}
